In [1]:
import psycopg2
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
connection = psycopg2.connect(
    host = 'XXX.XXX.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'XXX',
    database='XXX'
    )
cursor=connection.cursor()

In [3]:
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,FPIS,NAME,character varying,public
1,FPIS,FIPS_ID,integer,public
2,TEMPERATURE,DATE,date,public
3,TEMPERATURE,Country,character varying,public
4,TEMPERATURE,AverageTemperatureUncertaintyF,numeric,public
5,TEMPERATURE,STATE,character varying,public
6,TEMPERATURE,TEMPERATURE_ID,integer,public
7,TEMPERATURE,Fahrenheit,numeric,public
8,TEMPERATURE,AverageTemperatureF,numeric,public
9,USDisaster,pa_program_declared,integer,public


In [6]:
sql_USDisaster = """ SELECT * FROM public."USDisaster" ORDER BY declaration_date DESC """
disaster_df = pd.read_sql(sql_USDisaster, con=connection)
disaster_df

,fema_declaration,disaster_number,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,ih_program_declared,ia_program_declared,pa_program_declared,hm_program_declared,incident_begin_date,fips,place_code,designated_area,declaration_request_number,hash,last_refresh,id
0,FM-5416-CA,5416,CA,FM,2021-08-25,2021,Fire,French Fire,0,0,1,1,2021-08-24,6029,99029,Kern (County),21088,b2120fdada11e2ceefe3714927afe2443c719812,2021-08-25,61266e5e434e34d5eee6f25e
1,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6063,99063,Plumas (County),21087,d615785ddbaca2e0dbb66805d84246680c86f560,2021-08-25,6125a937434e34d5ee5cd8af
2,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6057,99057,Nevada (County),21087,dde89f65556f220034ac16cfa66abf7d135962da,2021-08-25,6125a937434e34d5ee5cd8c1
3,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6035,99035,Lassen (County),21087,74f77c93f158659bf1dd9f871227a9993244478d,2021-08-25,6125a937434e34d5ee5cd8a3
4,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6061,99061,Placer (County),21087,6e1ab1c91a9838148684d35413b598c5ea66df71,2021-08-25,6125a937434e34d5ee5cd8b6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61801,DR-5-MT,5,MT,DR,1953-06-06,1953,Flood,Floods,0,1,1,1,1953-06-06,30000,0,Statewide,53006,954449c15634fb45c8bea3ac975782793ccde050,2021-07-13,60c3b7a9a0ee349d71025774
61802,DR-4-MI,4,MI,DR,1953-06-02,1953,Tornado,Tornado,0,1,1,1,1953-06-02,26000,0,Statewide,53004,87a0c1dd5da249767f545e0c0a43f917e4e9ca83,2021-07-13,60c3b7a9a0ee349d7102577a
61803,DR-3-LA,3,LA,DR,1953-05-29,1953,Flood,Flood,0,1,1,1,1953-05-29,22000,0,Statewide,53005,4fb19699fdbba1387ffa2263fcc4a4e37a1de6d6,2021-07-13,60c3b7a9a0ee349d71025777
61804,DR-2-TX,2,TX,DR,1953-05-15,1953,Tornado,Tornado & Heavy Rainfall,0,1,1,1,1953-05-15,48000,0,Statewide,53003,c879557e78d059e6847e7688388c339d10f51979,2021-07-13,60c3b7a9a0ee349d71025783


In [37]:
# ignoring above SQL connection for now
temp_disaster_df = pd.read_csv("temp_disaster_merge_new.csv")
temp_disaster_df.columns

Index(['disaster_number', 'STATE', 'declaration_type', 'declaration_date',
       'fy_declared', 'incident_type', 'declaration_title', 'fips',
       'place_code', 'designated_area', 'declaration_request_number', 'hash',
       'last_refresh', 'id', 'year_x', 'month_x', 'month_year',
       'TEMPERATURE_ID', 'DATE', 'Country', 'Fahrenheit',
       'AverageTemperatureF', 'AverageTemperatureUncertaintyF', 'year_y',
       'month_y'],
      dtype='object')

In [40]:
temp_disaster_df.isnull().sum()

disaster_number                   0
STATE                             0
declaration_type                  0
declaration_date                  0
fy_declared                       0
incident_type                     0
declaration_title                 0
fips                              0
place_code                        0
designated_area                   0
declaration_request_number        0
hash                              0
last_refresh                      0
id                                0
year_x                            0
month_x                           0
month_year                        0
TEMPERATURE_ID                    0
DATE                              0
Country                           0
Fahrenheit                        0
AverageTemperatureF               0
AverageTemperatureUncertaintyF    0
year_y                            0
month_y                           0
dtype: int64

In [41]:
# drop columns we won't use in model
disaster_temp_concise = temp_disaster_df.drop(columns=["fy_declared", "declaration_date", "declaration_title", "hash", "last_refresh", "id", "month_year", 'TEMPERATURE_ID', 'DATE',
       'Country', 'Fahrenheit', 'AverageTemperatureUncertaintyF', 'year_y', 'month_y'])
disaster_temp_concise

,disaster_number,STATE,declaration_type,incident_type,fips,place_code,designated_area,declaration_request_number,year_x,month_x,AverageTemperatureF
0,1796,AK,DR,Severe Storm(s),2090,99090,Fairbanks North Star (Borough),8159,2008,9,40.5392
1,1796,AK,DR,Severe Storm(s),2068,99068,Denali (Borough),8159,2008,9,40.5392
2,1796,AK,DR,Severe Storm(s),2185,99185,North Slope (Borough),8159,2008,9,40.5392
3,1796,AK,DR,Severe Storm(s),2290,86650,Yukon Koyukuk Regional Educational Attendance ...,8159,2008,9,40.5392
4,1440,AK,DR,Earthquake,2170,99170,Matanuska-Susitna (Borough),2145,2002,11,20.1254
...,...,...,...,...,...,...,...,...,...,...,...
40727,2460,WY,FM,Fire,56023,99023,Lincoln (County),2126,2002,8,63.0392
40728,1268,WY,DR,Severe Ice Storm,56027,99027,Niobrara (County),99017,1999,2,28.6178
40729,1268,WY,DR,Severe Ice Storm,56015,99015,Goshen (County),99017,1999,2,28.6178
40730,2993,WY,FM,Fire,56001,99001,Albany (County),12050,2012,7,70.4534


In [42]:
disaster_temp_concise.dtypes

disaster_number                 int64
STATE                          object
declaration_type               object
incident_type                  object
fips                            int64
place_code                      int64
designated_area                object
declaration_request_number      int64
year_x                          int64
month_x                         int64
AverageTemperatureF           float64
dtype: object

In [43]:
# encode incident type
dt_encoded = pd.get_dummies(disaster_temp_concise, columns=["incident_type"])
dt_encoded.columns

Index(['disaster_number', 'STATE', 'declaration_type', 'fips', 'place_code',
       'designated_area', 'declaration_request_number', 'year_x', 'month_x',
       'AverageTemperatureF', 'incident_type_Coastal Storm',
       'incident_type_Dam/Levee Break', 'incident_type_Drought',
       'incident_type_Earthquake', 'incident_type_Fire',
       'incident_type_Fishing Losses', 'incident_type_Flood',
       'incident_type_Freezing', 'incident_type_Human Cause',
       'incident_type_Hurricane', 'incident_type_Mud/Landslide',
       'incident_type_Severe Ice Storm', 'incident_type_Severe Storm(s)',
       'incident_type_Snow', 'incident_type_Terrorist',
       'incident_type_Tornado', 'incident_type_Toxic Substances',
       'incident_type_Tsunami', 'incident_type_Typhoon',
       'incident_type_Volcano'],
      dtype='object')

In [45]:
# drop columns so only using hurrican data 
dt_hurricane = dt_encoded.drop(columns=[
       'incident_type_Coastal Storm',
       'incident_type_Dam/Levee Break', 'incident_type_Drought',
       'incident_type_Earthquake', 'incident_type_Fire',
       'incident_type_Fishing Losses', 'incident_type_Flood',
       'incident_type_Freezing', 'incident_type_Human Cause','incident_type_Mud/Landslide',
       'incident_type_Severe Ice Storm', 'incident_type_Severe Storm(s)',
       'incident_type_Snow', 'incident_type_Terrorist',
       'incident_type_Tornado', 'incident_type_Toxic Substances',
       'incident_type_Tsunami', 'incident_type_Typhoon',
       'incident_type_Volcano'])
dt_hurricane.head()

,disaster_number,STATE,declaration_type,fips,place_code,designated_area,declaration_request_number,year_x,month_x,AverageTemperatureF,incident_type_Hurricane
0,1796,AK,DR,2090,99090,Fairbanks North Star (Borough),8159,2008,9,40.5392,0
1,1796,AK,DR,2068,99068,Denali (Borough),8159,2008,9,40.5392,0
2,1796,AK,DR,2185,99185,North Slope (Borough),8159,2008,9,40.5392,0
3,1796,AK,DR,2290,86650,Yukon Koyukuk Regional Educational Attendance ...,8159,2008,9,40.5392,0
4,1440,AK,DR,2170,99170,Matanuska-Susitna (Borough),2145,2002,11,20.1254,0


In [46]:
dt_hurricane.dtypes

disaster_number                 int64
STATE                          object
declaration_type               object
fips                            int64
place_code                      int64
designated_area                object
declaration_request_number      int64
year_x                          int64
month_x                         int64
AverageTemperatureF           float64
incident_type_Hurricane         uint8
dtype: object

In [47]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dt_hurricane["STATE"] = le.fit_transform(dt_hurricane["STATE"])
dt_hurricane["declaration_type"] = le.fit_transform(dt_hurricane["declaration_type"])
dt_hurricane["place_code"] = le.fit_transform(dt_hurricane["place_code"])
dt_hurricane["designated_area"] = le.fit_transform(dt_hurricane["designated_area"])


In [48]:
dt_hurricane.head()

,disaster_number,STATE,declaration_type,fips,place_code,designated_area,declaration_request_number,year_x,month_x,AverageTemperatureF,incident_type_Hurricane
0,1796,0,0,2090,216,564,8159,2008,9,40.5392,0
1,1796,0,0,2068,201,477,8159,2008,9,40.5392,0
2,1796,0,0,2185,269,1227,8159,2008,9,40.5392,0
3,1796,0,0,2290,154,1926,8159,2008,9,40.5392,0
4,1440,0,0,2170,260,1068,2145,2002,11,20.1254,0


In [49]:
dt_hurricane.dtypes

disaster_number                 int64
STATE                           int64
declaration_type                int64
fips                            int64
place_code                      int64
designated_area                 int64
declaration_request_number      int64
year_x                          int64
month_x                         int64
AverageTemperatureF           float64
incident_type_Hurricane         uint8
dtype: object

In [50]:
dt_hurricane.isnull().sum()

disaster_number               0
STATE                         0
declaration_type              0
fips                          0
place_code                    0
designated_area               0
declaration_request_number    0
year_x                        0
month_x                       0
AverageTemperatureF           0
incident_type_Hurricane       0
dtype: int64

In [51]:
len(dt_hurricane["disaster_number"])

40732

In [52]:
# split into X and Y valuese
y = dt_hurricane["incident_type_Hurricane"]
X = dt_hurricane.drop(columns=["incident_type_Hurricane"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [53]:
# scale and normalize data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_scaler  = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [54]:
# instantate Logistic Regression
classifier = LogisticRegression(random_state=1)

In [55]:
# fit model
classifier.fit(X_train_scaled, y_train)


LogisticRegression(random_state=1)

In [56]:
#predict
y_predict = classifier.predict(X_test_scaled)

In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.8999312579789845

In [58]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,7833,460
Actual 1,559,1331
